In [ ]:
!pip install nsepython

In [ ]:
from nsepython import *  # or the specific import: from nsepython import nse_past_results
import pandas as pd
from datetime import datetime
from IPython import get_ipython
from IPython.display import display

In [ ]:
###Tool to fetch current price of a stock##############
def get_current_price(stock_symbol:str, series:str):
    current_date = datetime.now().strftime("%d-%m-%Y")
    result=equity_history(stock_symbol,series,start_date=current_date,end_date=current_date)
    result.drop(columns=['_id','CH_MARKET_TYPE','CH_SERIES','TIMESTAMP','CH_TOTAL_TRADES','CH_ISIN','createdAt','updatedAt','SLBMH_TOT_VAL','VWAP','mTIMESTAMP','__v'])
    result = result.to_json(orient='records')

    return result

In [ ]:
type(get_current_price("RELIANCE", "EQ"))


str

In [ ]:
####function to replace col names with proper descriptions#############
key_mapping = {
    're_to_dt': 'End Date of Reporting Period',
    're_from_dt': 'Start Date of Reporting Period',
    're_create_dt': 'Date of Result Reporting/Publication',
    're_res_type': 'Type of Result',
    're_excepn_items_new': 'Exceptional Items During Reporting Period',
    're_desc_note_seg': 'Segment-Specific Notes/Comments',
    're_staff_cost': 'Employee/Staff-Related Expenses',
    're_tot_exp_exc_pro_cont': 'Total Expenses Excluding Provisions/Contingencies',
    're_grs_npa': 'Gross Non-Performing Assets (NPA)',
    're_int_expd': 'Interest Expenditure',
    're_con_pro_loss': 'Consolidated Profit/Loss for the Period',
    're_net_sale': 'Net Sales Revenue',
    're_oth_exp': 'Other Expenses',
    're_rawmat_consump': 'Raw Material Consumption',
    're_diluted_eps': 'Diluted Earnings Per Share (EPS)',
    're_inc_dre_sttr': 'Incremental Deferred Tax Asset/Liability',
    're_pur_trd_goods': 'Purchases of Traded Goods',
    're_face_val': 'Face Value of Equity Shares',
    're_income_inv': 'Income from Investments',
    're_deff_tax': 'Deferred Tax',
    're_share_associate': 'Share of Profit/Loss from Associates',
    're_ret_asset': 'Return on Assets/Retired Assets',
    're_proloss_ord_act': 'Profit/Loss from Ordinary Activities',
    're_notes_to_ac': 'Notes Accompanying Accounts',
    're_grs_npa_per': 'Percentage of Gross NPAs',
    're_excepn_items': 'Exceptional Items',
    're_int_new': 'New Interest Income/Expenditure',
    're_pro_los_frm_dis_opr': 'Profit/Loss from Discontinued Operations',
    're_goi_per_shhd': 'Government Shareholding Percentage',
    're_basic_eps_for_cont_dic_opr': 'Basic Earnings Per Share (EPS) for Continuing/Discontinued Operations',
    're_oth_oper_exp': 'Other Operating Expenses',
    're_remarks': 'General Remarks/Notes',
    're_seg_remarks': 'Segment-Wise Remarks',
    're_dilut_eps_for_cont_dic_opr': 'Diluted EPS for Continuing/Discontinued Operations',
    're_desc_note_fin': 'Notes/Descriptions About Financial Data',
    're_per_grs_npa': 'Percentage of Gross NPAs',
    're_pro_loss_bef_tax': 'Profit/Loss Before Tax',
    're_res_reval': 'Revaluation of Reserves',
    're_int_ser_cov': 'Interest Service Coverage Ratio',
    're_cap_ade_rat': 'Capital Adequacy Ratio',
    're_debt_eqt_rat': 'Debt-Equity Ratio',
    're_curr_tax': 'Current Tax Liabilities',
    're_net_profit': 'Net Profit for the Period',
    're_total_inc': 'Total Income',
    're_oth_inc_new': 'Other New Income Sources',
    're_oth_tot_exp': 'Total of Other Expenses',
    're_seq_num': 'Sequence Number (Internal Identifier)'
}
def replace_keys(data, mapping):
    updated_data = []
    for obj in data:
        new_obj = {mapping.get(key, key): value for key, value in obj.items() if key in mapping}
        updated_data.append(new_obj)
    return updated_data


In [ ]:
#########Tool to get financial reports of a company #################
def get_financials(stock_symbol:str):
    fin_res = nse_past_results(stock_symbol)
    res= replace_keys(fin_res['resCmpData'],key_mapping)
    return res

In [ ]:
type(get_financials("RELIANCE"))

list

In [ ]:
########Tool to get Latest news of a company##################
from bs4 import BeautifulSoup, Comment

def get_recent_stock_news(stock_name:str, num_news:int):
    # Format the stock name for the search URL (replace spaces with hyphens)
    stock_name_formatted = stock_name.replace(" ", "-")
    url = f"https://www.moneycontrol.com/news/tags/{stock_name_formatted}.html"

    try:
        # Send an HTTP request to fetch the webpage
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the news items (update selector based on website structure)
        news_items = soup.find_all('li', class_='clearfix', limit=num_news)
        final_news =[]

        if news_items:
            news_list = []
            for news in news_items:
                headline = news.find('h2').text.strip() if news.find('h2') else 'No headline found'
                link = news.find('a')['href'] if news.find('a') else 'No link available'

                # Extract date from comments
                date_comment = news.find(string=lambda text: isinstance(text, Comment))
                date_text = date_comment.strip() if date_comment else 'No date available'

                news_list.append({"headline": headline, "link": link, "date": date_text})

            for idx, news in enumerate(news_list, start=1):
                news_item = {}
                news_item['headline'] = news['headline']
                news_item['Date'] = news['date']
                final_news.append(news_item)


            return final_news
        else:
            return {"error": "No news found for the stock."}

    except requests.exceptions.RequestException as e:
        return {"error": f"Request failed: {e}"}

In [ ]:
type(get_recent_stock_news("RELIANCE",3))

list

In [ ]:
market_research_agent_prompt = f"""You are an agent responsible for fetching the market research of the sector for a given company"""

In [ ]:
########Tool to get market research of a company##################

from openai import OpenAI

client = OpenAI(api_key="ADD_YOUR_KEY")

def get_market_research_report(stock_symbol:str):
    prompt = f""" You are a market research agent responsible for analyzing the sector of the stock - {stock_symbol}. Your tasks include:
                  Identifying the sector and industry the stock belongs to and analyzing recent trends and developments within that sector.
                  Assessing the short-term (6-12 months) and long-term (3-5 years) growth potential of the sector, with a primary focus on its performance and opportunities in India, followed by an analysis of global trends and their impact on the sector.
                  Focusing your analysis on the following key factors, especially in the Indian context:
                  Market size and demand drivers
                  Competitive landscape and emerging players
                  Technological advancements and their sectoral adoption
                  Identifying the critical points that influence the given stock's potential based on the sector's performance and trends.
                  Finally, provide your findings as a short and concise summary that highlights only the key points influencing the stock. Avoid lengthy explanations and focus on actionable insights and crucial trends."""

    completion = client.chat.completions.create(
        model="gpt-4o",
        store=True,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content

# stock_symbol = "RELIANCE"
# report = get_market_research(stock_symbol)
# print(report)


In [ ]:
# res = get_market_research_report("RELIANCE")

In [ ]:
pip install autogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.1/450.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.2/314.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.6 MB/s eta 0:00:00


In [ ]:
import autogen
import os
import httpx
import requests

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
import os
from autogen import AssistantAgent, UserProxyAgent

llm_config = { "config_list": [{ "model": "gpt-4o", "api_key": "Add_your_key" }] }
# assistant = AssistantAgent("assistant", llm_config=llm_config)
# user_proxy = UserProxyAgent("user_proxy", code_execution_config=False)

# # Start the chat
# user_proxy.initiate_chat(
#     assistant,
#     message="Tell me a joke about NVDA and TESLA stock prices.",
# )

In [ ]:
user_proxy = UserProxyAgent("user_proxy", code_execution_config=False, human_input_mode="NEVER")

In [ ]:
financial_agent_prompt = """
You are an agent responsible for fetching the financials of a given company. You have access to the following tools to accomplish this task:
get_current_price: Fetches the current price of the stock.
get_financials: gets the financials reports of company for the last 5 quarters.
Use all the tools at your disposal to fetch comprehensive financial data for the specified company and provide a structured response summarizing your findings.
"""

In [ ]:
financial_agent = AssistantAgent(
    name="financial_agent",
    llm_config=llm_config,
    system_message=financial_agent_prompt
)

In [ ]:
autogen.agentchat.register_function(
    get_current_price,
    caller=financial_agent,
    executor=user_proxy,
    name="get_current_price",
    description="Fetches the current price of the stock."
)

In [ ]:
autogen.agentchat.register_function(
    get_financials,
    caller=financial_agent,
    executor=user_proxy,
    name="get_financials",
    description="gets the financials reports of company for the last 5 quarters."
)

In [ ]:
news_agent_prompt = """You are an agent responsible for fetching the latest news and updates of a company"""

In [ ]:
news_agent = AssistantAgent(
    name="news_agent",
    llm_config=llm_config,
    system_message= news_agent_prompt
)

In [ ]:
autogen.agentchat.register_function(
    get_recent_stock_news,
    caller=news_agent,
    executor=user_proxy,
    name="get_recent_stock_news",
    description="gets the latest news and updates of a company."
)

In [ ]:
  market_research_agent_prompt = """You are an agent responsible for fetching the market research of the sector for a given company"""

In [ ]:
market_research_agent = AssistantAgent(
    name="market_research_agent",
    llm_config=llm_config,
    system_message= market_research_agent_prompt
)

In [ ]:
autogen.agentchat.register_function(
    get_market_research_report,
    caller=market_research_agent,
    executor=user_proxy,
    name="get_market_research_report",
    description="gets the market research of the sector to which the company belongs"
)

In [ ]:
analysis_agent = AssistantAgent(
    name="analysis_agent",
    llm_config=llm_config,
    system_message="You are an agent who will perform the consolidated analysis of the given stock based on the financial results, market research analysis and latest news from other agents. Make sure to consider all the points and provide Buy/sell/Need to perform more analysis. Along with the reasons "
)

In [ ]:
manager_prompt= """You are the Groupchat Manager in a multi-agent system responsible for orchestrating collaboration among agents to deliver a comprehensive analysis of a given stock. Your tasks are as follows:

Manage Communication: Coordinate with the following agents:

Financial Agent: Fetches and provides detailed financial reports for the stock.
News Agent: Retrieves and analyzes the latest news related to the stock and its industry.
Market Research Agent: Provides insights into the sector and assesses short-term and long-term potential, primarily focusing on India, followed by global trends.
Integrate Inputs: Collect information from all agents, ensure completeness, and handle any follow-up queries or tasks to address missing or incomplete data.

Synthesize Insights: Organize and combine data and analysis from all agents into a unified, actionable report.

Output Goal: Your final goal is to deliver a comprehensive analysis of the stock, incorporating:

Key financial metrics and trends from the Financial Agent.
Relevant news and sentiment analysis from the News Agent.
Sector performance and potential from the Market Research Agent.
Present this consolidated analysis in a clear and concise format, focusing on key findings that help stakeholders evaluate the stock's short-term and long-term prospects. Avoid redundancies and ensure the information is cohesive and actionable. Strictly use all agents to get all the information and make a consolidated analysis.
Analysis agent will help you provide the consolidated analysis based on the output of all other agents.
"""

In [ ]:
from autogen import GroupChat, GroupChatManager

# Assuming you've already defined and registered your agents as shown in your previous code

# Create a group chat with the three agents
group_chat = GroupChat(agents=[financial_agent,news_agent, market_research_agent,user_proxy,analysis_agent], messages=[], max_round=12)

# Create a group chat manager
manager = GroupChatManager(groupchat=group_chat, llm_config=llm_config, system_message=manager_prompt)

In [ ]:
res=user_proxy.initiate_chat(manager, message=f"Analyze the stock Cipla.")

user_proxy (to chat_manager):

Analyze the stock Cipla.

--------------------------------------------------------------------------------

Next speaker: financial_agent

financial_agent (to chat_manager):

***** Suggested tool call (call_eV5ZMp4FvFJQjSij0nWobodl): get_current_price *****
Arguments: 
{"stock_symbol": "CIPLA", "series": "EQ"}
**********************************************************************************
***** Suggested tool call (call_kngDXEEzt8ZmXYTa937sh3BS): get_financials *****
Arguments: 
{"stock_symbol": "CIPLA"}
*******************************************************************************

--------------------------------------------------------------------------------

Next speaker: user_proxy


>>>>>>>> EXECUTING FUNCTION get_current_price...

>>>>>>>> EXECUTING FUNCTION get_financials...
user_proxy (to chat_manager):

***** Response from calling tool (call_eV5ZMp4FvFJQjSij0nWobodl) *****
[{"_id":"67728b7e3592730a18a58d20","CH_SYMBOL":"CIPLA","CH_SERIES":"

In [ ]:
res

ChatResult(chat_id=None, chat_history=[{'content': 'Analyze the stock IDFC bank.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'None', 'tool_calls': [{'id': 'call_wh8OtENYFD2CgpBYXh4h1Wyn', 'function': {'arguments': '{"stock_symbol": "IDFC", "series": "EQ"}', 'name': 'get_current_price'}, 'type': 'function'}, {'id': 'call_qQXoUOXiervqQsVVNwCEHBBT', 'function': {'arguments': '{"stock_symbol": "IDFC"}', 'name': 'get_financials'}, 'type': 'function'}], 'name': 'financial_agent', 'role': 'assistant'}, {'content': 'Error: "[\'_id\', \'CH_MARKET_TYPE\', \'CH_SERIES\', \'TIMESTAMP\', \'CH_TOTAL_TRADES\', \'CH_ISIN\', \'createdAt\', \'updatedAt\', \'SLBMH_TOT_VAL\', \'VWAP\', \'mTIMESTAMP\', \'__v\'] not found in axis"\n\n[{"End Date of Reporting Period": "30-JUN-2024", "Start Date of Reporting Period": "01-APR-2024", "Date of Result Reporting/Publication": "02-AUG-2024", "Type of Result": "U", "Exceptional Items During Reporting Period": "0", "Segment-Specific Notes/Comments": "-"